# Example: Downloading datasets with the `ecmwfapi`

**Note:** Any changes you make to this notebook will be lost once the page is closed or refreshed. Please download any files you would like to keep.

**Note:** The WASM-based version of the compression lab running inside JupyterLite has only been tested in recent Firefox browsers.

In [1]:
# TODO: move this into the lab-specific patches
try:
    import js
    import pyodide_http
    import urllib
except ImportError:
    pass
else:
    _old_send = pyodide_http._core.send

    def _new_send(
        request: pyodide_http._core.Request, stream: bool = False
    ) -> pyodide_http._core.Response:
        if js.URL.new(request.url).origin != js.location.origin:
            request.url = "https://proxy.climet.eu/" + request.url
            resp = _old_send(request, stream)
            if resp.status_code in [251, 252, 253, 257, 258]:
                resp.status_code += 50
            return resp
        return _old_send(request, stream)

    pyodide_http._core.send = _new_send

    def _new_urlopen(url, *args, **kwargs):
        import urllib.request
        from http.client import HTTPResponse

        from pyodide_http._core import Request, send
        from pyodide_http._urllib import FakeSock

        method = "GET"
        data = None
        headers = {}
        if isinstance(url, urllib.request.Request):
            method = url.get_method()
            data = url.data
            headers = dict(url.header_items())
            url = url.full_url

        request = Request(method, url, headers=headers, body=data)
        resp = send(request)

        # Build a fake http response
        # Strip out the content-length header. When Content-Encoding is 'gzip' (or other
        # compressed format) the 'Content-Length' is the compressed length, while the
        # data itself is uncompressed. This will cause problems while decoding our
        # fake response.
        headers_without_content_length = (
            {k: v for k, v in resp.headers.items() if k != "content-length"}
            if "content-encoding" in resp.headers.keys()
            else resp.headers
        )
        response_data = (
            b"HTTP/1.1 "
            + str(resp.status_code).encode("ascii")
            + b"\n"
            + "\n".join(
                f"{'_'.join(k.title() for k in key.split('_'))}: {value}"
                for key, value in headers_without_content_length.items()
            ).encode("ascii")
            + b"\n\n"
            + resp.body
        )

        response = HTTPResponse(FakeSock(response_data))
        response.begin()
        return response

    def _new_urlopen_self_removed(self, url, *args, **kwargs):
        return _new_urlopen(url, *args, **kwargs)

    urllib.request.urlopen = _new_urlopen
    urllib.request.OpenerDirector.open = _new_urlopen_self_removed

## Installing and configuring the `ecmwfapi` package

In [2]:
%pip install ecmwf-api-client
import ecmwfapi

To run this example, you need an API key available from https://api.ecmwf.int/v1/key/, which looks as follows:

```json
{
    "url"   : "<ECMWF-API-URL>",
    "key"   : "<ECMWF-API-KEY>",
    "email" : "<ECMWF-API-EMAIL>"
}
```

Please enter these values to set the `ECMWF_API_*` environment variables in the next cell.

Please remember that you must keep your API key private and should not share a notebook that contains your API key.

In [3]:
import os
os.environ["ECMWF_API_URL"] = "<ECMWF-API-URL>"
os.environ["ECMWF_API_KEY"] = "<ECMWF-API-KEY>"
os.environ["ECMWF_API_EMAIL"] = "<ECMWF-API-EMAIL>"

## Retrieving the dataset

You can now retrieve the following example GRIB dataset. Once the download has completed, you should have a new `data.grib` file in your local directory.

In [4]:
from ecmwfapi import ECMWFDataServer

server = ECMWFDataServer()
server.retrieve(
    {
        "dataset": "tigge",
        "step": "24",
        "number": "all",
        "levtype": "sl",
        "date": "20071001",
        "time": "00",
        "origin": "all",
        "type": "pf",
        "param": "tp",
        "area": "70/-130/30/-60",
        "grid": "2/2",
        "target": "data.grib",
    }
)

2024-08-30 12:29:53 ECMWF API python library 1.6.0
2024-08-30 12:29:53 ECMWF API at https://api.ecmwf.int/v1
2024-08-30 12:29:56 Welcome
2024-08-30 12:29:57 In case of problems, please check https://confluence.ecmwf.int/display/WEBAPI/Web+API+FAQ or contact servicedesk@ecmwf.int
2024-08-30 12:29:57 ------------ WARNING ------------
2024-08-30 12:29:57 Access to this dataset is transitioning to a new interface, dates to be announced soon
2024-08-30 12:29:57 For more information on how to access this data in the future, visit https://confluence.ecmwf.int/x/-wUiEw
2024-08-30 12:29:57 ---------------------------------
2024-08-30 12:29:57 Request submitted
2024-08-30 12:29:57 Request id: 66d19115c7953e889e127628
2024-08-30 12:29:57 Request is submitted
2024-08-30 12:29:59 Request is active
Calling 'nice mars /tmp/20240830-0920/7e/tmp-_mars-bxZJS9.req'
Forcing MIR_CACHE_PATH=/data/ec_coeff
mars - WARN -
mars - WARN -
MIR environment variables:
MIR_CACHE_PATH=/data/ec_coeff
MIR_LSM_NAMED=1km.